### Synchronous VS Asynchronous Clients


Given the growing demand for real-time applications and user demands for instant responses, it's crucial to grasp the performance implications between Sync and Async clients. In this notebook, we'll delve into the variations between asynchronous and synchronous response times using UnifyAI's API.

In order to run this notebook, you need to generate your UNIFY key from the [console](https://console.unify.ai/login?callbackUrl=%2F). Once you have it, assign it to the `UNIFY_KEY` variable below.

In [ ]:
UNIFY_KEY=#ENTERUNIFYKEY

#### Install Dependencies

To run this notebook, you will need to install the `unifyai` [python package](https://pypi.org/project/unifyai/). You can do so by running the cell below ⬇️

In [ ]:
!pip install unifyai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.3/268.3 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.6 MB/s eta 0:00:00


#### Synchronous Clients

A Synchronous client handles requests sequentially, processing one at a time. This means that each request must be fully handled before the next one is processed, resulting in potential blocking of the program's execution. You can use a Sync client with Unify as shown below:

In [ ]:
from unify import Unify
unify = Unify(
    api_key=UNIFY_KEY,
    endpoint="llama-2-13b-chat@anyscale"
)
response = unify.generate(user_prompt="Hello Llama! Who was Isaac Newton?")
print(response)

HOO-RAY! *tutu* Hello there, young grasshopper! *bray* Isaac Newton was a majestic figure known for his work in math and science. He was born in 1642 in England and grew up to be a brilliant thinker and a fellow of the Royal Society. *twirl*

Newton is perhaps best known for his laws of motion and his law of universal gravitation. These laws explained how objects move and how gravity works. He also discovered calculus, which is a way of using math to understand how things change over time. *mathematical mnum hop*

But that's not all, oh no! Newton was also a bit of an alchemist and studied the nature of light. He even invented a fancy piece of equipment called a "reflecting telescope" to observe the heavens. *shimmer*

Newton was a true renaissance thinker, and his contributions to science and mathematics are still celebrated today. *tutu* He was a true llama of learning, and his legacy continues to inspire us all. *bray*


#### Async Clients

An Asynchronous client can handle multiple requests concurrently without blocking. To use the Async client, import `AsyncUnify` instead of `Unify` and use `await` with the `.generate` method. This signals to the program to pause execution until the asynchronous operation completes. Additionally, we'll use the `asyncio` library for managing asynchronous execution and coroutines. `asyncio` provides tools for building concurrent programs using coroutines, which can be paused and resumed, allowing for efficient handling of asynchronous tasks.


NOTE: Running ansyncio in notebooks conflicts with the existing event loop run in the notebook. As a workaround, we need to use `net_asyncio.apply`. Please see [this issue](https://github.com/jupyter/notebook/issues/3397) for more details.

In [ ]:
from unify import AsyncUnify
import asyncio
import nest_asyncio
nest_asyncio.apply()

async_unify = AsyncUnify(
   api_key=UNIFY_KEY,
   endpoint="llama-2-13b-chat@anyscale"
)

async def main():
   responses = await async_unify.generate(user_prompt="Hello Llama! Who was Isaac Newton?")
   print(responses)

asyncio.run(main())

HEYA HUMAN! *bleshes* Oh, you're talkin' 'bout Sir Isaac Newton, the famous English mathematician and physicist? *noms on some grass* He lived in the 17th and 18th centuries, and is known for his laws of motion and universal gravitation. *giggles* He was a pretty smart dude, if I do say so myself! *flaunts my banned-from-the-library-for-life status*

But enough about me, let's talk about Newton! *grin* He was born in Woolsthorpe, Lincolnshire, England in 1643, and grew up to be a brilliant mathematician and physicist. He studied at Trinity College in Cambridge, and later became a professor there. *nbd*

Newton's most famous contributions to science are his three laws of motion, which describe how objects move and respond to forces. He also developed the Law of Universal Gravitation, which states that every object in the universe attracts every other object with a force proportional to the product of their masses and inversely proportional to the square of the distance between them. *ge

Now, our goal is to compare the response times of synchronous vs asynchronous clients when handling multiple requests. Let's start by defining some helper functions.



In [ ]:
async def send_async_request(user_prompt):
    """
    Uses an Async client to generate the response for the user_prompt.

    Parameters:
        user_prompt (str): The prompt provided by the user.

    Returns:
        str: The response generated.
    """
    response = await async_unify.generate(user_prompt=user_prompt)
    return response

def send_sync_request(user_prompt):
    """
    Uses a sync client to generate the response for the user_prompt.

    Parameters:
        user_prompt (str): The prompt provided by the user.

    Returns:
        str: The response generated.
    """
    response = unify.generate(user_prompt=user_prompt)
    return response


We'll create two functions to send multiple requests to asynchronous and synchronous clients, respectively, and measure their processing time. For the synchronous client, requests will be sent sequentially in a loop, while for the asynchronous client, we'll utilize `asyncio.gather` to execute multiple requests concurrently.


In [ ]:
import time
import asyncio

async def run_async_requests(num_requests):
    """
    Runs multiple asynchronous requests for generating responses based on a user prompt and measures the time taken.

    Parameters:
        num_requests (int): The number of requests to be sent.

    Returns:
        float: The total time taken to process all requests.
    """
    user_prompt = "Hello! Tell me your favorite physics fact!"
    start = time.time()
    _ = await asyncio.gather(*(send_async_request(user_prompt) for _ in range(num_requests)))
    end = time.time()
    return end - start

def run_sync_requests(num_requests):
    """
    Runs multiple synchronous requests for generating responses based on a user prompt and measures the time taken.

    Parameters:
        num_requests (int): The number of requests to be sent.

    Returns:
        float: The total time taken to process all requests.
    """
    user_prompt = "Hello! Tell me your favorite physics fact!"
    start = time.time()
    _ = [send_sync_request(user_prompt) for _ in range(num_requests)]
    end = time.time()
    return end - start


Now, let's measure the time taken by each client for 10 requests.

In [ ]:
    num_requests = 10
    # Send asynchronous requests
    async_response_times = asyncio.run(run_async_requests(num_requests))
    # Print response times
    print("Asynchronous Response Times:", async_response_times)
    # Send synchronous requests
    sync_response_times = run_sync_requests(num_requests)
    print("Synchronous Response Times:", sync_response_times)

Asynchronous Response Times: 8.351824045181274
Synchronous Response Times: 55.45608472824097


As expected, the Asynchronous client peforms much better than the sequential synchorous client.

### Round Up
 Congratulations! 🚀 You now have an understanding of Async and Sync clients and can hopefully leverage these concepts in your own applications. If you're a contributor and want to earn a cool badge, you should fill out this [form](https://docs.google.com/forms/d/e/1FAIpQLSfQHh2H08G5UeR2s3Vc5zYTpnuLGGKAcW3pT7RliUzsytxUug/viewform?usp=sf_link) which contains a few quiz questions about the concepts you learnt in this tutorial. Happy quizzing!

